# Extended RNA-Seq Analysis Training Demo

## Overview

For simplicity and time, The short tutorial workflow uses only partial run data from the Cushman et al., project.

The tutorial repeats the short tutorial, but with extended steps.

This tutorial demonstrates how to run an RNA-Seq workflow using a prokaryotic data set. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to quantitate gene expression. Extended steps include using SRA tools to download the full dataset, joining gene count results, and formatting those results for use in R for further downstream analysis.

For those seeking practice, we encourage you to create your own notebook and attempt to repeat through the full work flow yourself from scratch, using this extended tutorial as a reference as needed.

![RNA-Seq workflow](images/rnaseq-workflow.png)

### STEP 1: Install Mambaforge and then install snakemake using bioconda.

First install Mambaforge.


In [ ]:
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
!bash Mambaforge-$(uname)-$(uname -m).sh -b -u -p $HOME/mambaforge
!export PATH="$HOME/mambaforge/bin:$PATH"

Next, using mambaforge and bioconda, install the tools that will be used in this tutorial.

In [ ]:
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda trimmomatic
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda fastqc
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda multiqc
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda salmon
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda sra-tools
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda parallel-fastq-dump
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda -c defaults entrez-direct

### STEP 2: Setup Environment

Create a set of directories to store the reads, reference sequence files, and output files.


In [ ]:
!cd $HOMEDIR
!echo $PWD
!mkdir -p data
!mkdir -p data/raw_fastq
!mkdir -p data/trimmed
!mkdir -p data/fastqc
!mkdir -p data/aligned
!mkdir -p data/reference

### STEP 3: Downloading relevant FASTQ files using SRA Tools

Next we will need to download the relevant fastq files.

The sequence data for this tutorial comes from work by Cushman et al., <em><a href='https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8191103/'>Increased whiB7 expression and antibiotic resistance in Mycobacterium chelonae carrying two prophages</a><em>.

We will be downloading the sample runs from this project using SRA tools, downloading from the NCBI's SRA (Sequence Run Archives).


### STEP 3.1: Finding run accession numbers.

The SRA stores sequence data in terms of runs, (SRR stands for Sequence Read Run). To download runs, we will need the accession ID for each run we wish to download. 

The Cushman et al., project contains 12 runs. To make it easier, these are the run IDs associated with this project:

SRR13349122
SRR13349123
SRR13349124
SRR13349125
SRR13349126
SRR13349127
SRR13349128
SRR13349129
SRR13349130
SRR13349131
SRR13349132
SRR13349133


In this case, all these runs belong to the SRP (Sequence Run Project): SRP300216.

Sequence run experiments can be searched for using the SRA database on the NCBI website; and article specific sample run information can be found in the supplementary section of that article.

For instance, here, the the authors posted a link to the sequence data GSE (Gene Series number), <a href='https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE164210'>GSE164210</a>. This leads to the appropriate 'Gene Expression Omnibus' page where, among other useful files and information, the relevant SRA database link can be found. 

### STEP 3.2: Using the SRA-toolkit.

Now use the Sequence Run accession ID to download the sequence data.

In [7]:
!prefetch  -O data/raw_fastq/ SRR13349122


2022-03-08T00:45:13 prefetch.2.11.0: 1) Downloading 'SRR13349122'...
2022-03-08T00:45:13 prefetch.2.11.0:  Downloading via HTTPS...
2022-03-08T00:45:31 prefetch.2.11.0:  HTTPS download succeed
2022-03-08T00:45:33 prefetch.2.11.0:  'SRR13349122' is valid
2022-03-08T00:45:33 prefetch.2.11.0: 1) 'SRR13349122' was downloaded successfully
2022-03-08T00:45:33 prefetch.2.11.0: 'SRR13349122' has 0 unresolved dependencies


Notice the SRA archives sequence files in the SRA format. 

Typically genome workflows processed data in the form of zipped or unzipped .fastq, or .fasta files

So before we move on, we need to convert the files from .sra to .fastq using the fastq-dump tool:

In [10]:
#paired-end reads use the --split-files flag.
#to save space, convert to a zipped fastq file with the --gzip flag.
!fastq-dump -O data/raw_fastq/ --split-files --gzip data/raw_fastq/SRR13349122

Read 10827590 spots for data/raw_fastq/SRR13349122
Written 10827590 spots for data/raw_fastq/SRR13349122


### STEP 3.3 Downloading multiple files using the SRA-toolkit.

One may, as in our case, wish to download multiple runs at once.

To aid in this, SRA-tools supports batch downloading.

We can download multiple SRA files using a single line of code by creating a list of the SRA IDs we wish to download, and inputting that into the prefetch command.

For instance, one can first create a list of the 12 accession IDs for this project however they wish:

In [11]:
!for i in {22..33}; do echo "SRR133491$i"; done > data/raw_fastq/list_of_accesionIDS.txt
!cat data/raw_fastq/list_of_accesionIDS.txt

SRR13349122
SRR13349123
SRR13349124
SRR13349125
SRR13349126
SRR13349127
SRR13349128
SRR13349129
SRR13349130
SRR13349131
SRR13349132
SRR13349133


And then feed that list into the sra-toolkit prefetch command

In [12]:
!prefetch -O data/raw_fastq/ --option-file data/raw_fastq/list_of_accesionIDS.txt


2022-03-08T01:04:26 prefetch.2.11.0: 1) 'SRR13349122' is found locally
2022-03-08T01:04:26 prefetch.2.11.0: 'SRR13349122' has 0 unresolved dependencies

2022-03-08T01:04:26 prefetch.2.11.0: 2) Downloading 'SRR13349123'...
2022-03-08T01:04:26 prefetch.2.11.0:  Downloading via HTTPS...
2022-03-08T01:04:45 prefetch.2.11.0:  HTTPS download succeed
2022-03-08T01:04:46 prefetch.2.11.0:  'SRR13349123' is valid
2022-03-08T01:04:46 prefetch.2.11.0: 2) 'SRR13349123' was downloaded successfully
2022-03-08T01:04:46 prefetch.2.11.0: 'SRR13349123' has 0 unresolved dependencies

2022-03-08T01:04:46 prefetch.2.11.0: 3) Downloading 'SRR13349124'...
2022-03-08T01:04:46 prefetch.2.11.0:  Downloading via HTTPS...
2022-03-08T01:05:07 prefetch.2.11.0:  HTTPS download succeed
2022-03-08T01:05:08 prefetch.2.11.0:  'SRR13349124' is valid
2022-03-08T01:05:08 prefetch.2.11.0: 3) 'SRR13349124' was downloaded successfully
2022-03-08T01:05:08 prefetch.2.11.0: 'SRR13349124' has 0 unresolved dependencies

2022-03-08

### STEP 3.3 Converting Multiple SRA files to Fastq

Fastq-dump does not have native batch compatibility.

There are various ways to get around this, for instance, commonly by using bash pipes or for-loops.

One can also attempt to speed up the process of converting SRA to fastq files, by using fasterq-dump from the SRA-Tools library, or 'parallel-fastq-dump' from the separate parralel-fastq-dump library.

There is more than one right way to do anything, but parallel-fastq-dump has the benefit of automatic zipping, whereas fasterq-dump does not have --gzip flag.

Below is an example of one way to batch convert SRA to fastq files, piping in the same accession list used to download the SRAs with prefetech.

In [14]:
!cat data/raw_fastq/list_of_accesionIDS.txt |xargs -I {} parallel-fastq-dump -O data/raw_fastq/ --tmpdir . --threads 8 --gzip --split-files --sra-id {}

2022-03-08 01:12:14,434 - SRR ids: ['SRR13349122']
2022-03-08 01:12:14,434 - extra args: ['--gzip', '--split-files']
2022-03-08 01:12:14,434 - tempdir: ./pfd_6j6anfig
2022-03-08 01:12:14,434 - CMD: sra-stat --meta --quick SRR13349122
2022-03-08 01:12:15,307 - SRR13349122 spots: 10827590
2022-03-08 01:12:15,307 - blocks: [[1, 1353448], [1353449, 2706896], [2706897, 4060344], [4060345, 5413792], [5413793, 6767240], [6767241, 8120688], [8120689, 9474136], [9474137, 10827590]]
2022-03-08 01:12:15,308 - CMD: fastq-dump -N 1 -X 1353448 -O ./pfd_6j6anfig/0 --gzip --split-files SRR13349122
2022-03-08 01:12:15,309 - CMD: fastq-dump -N 1353449 -X 2706896 -O ./pfd_6j6anfig/1 --gzip --split-files SRR13349122
2022-03-08 01:12:15,310 - CMD: fastq-dump -N 2706897 -X 4060344 -O ./pfd_6j6anfig/2 --gzip --split-files SRR13349122
2022-03-08 01:12:15,312 - CMD: fastq-dump -N 4060345 -X 5413792 -O ./pfd_6j6anfig/3 --gzip --split-files SRR13349122
2022-03-08 01:12:15,314 - CMD: fastq-dump -N 5413793 -X 6767

### STEP 4: Copy reference transcriptome files that will be used by Salmon using E-Direct

Salmon is a tool that aligns RNA-Seq reads to a set of transcripts rather than the entire genome.

We will need to download a transcript reference from the NCBI database. There are several ways to do this. One way is to search through the NCBI database and download using the FTP link. Another way is to use the NCBI entrez-direct tool. 

The most current assembly for Mycobacteroides chelonae is ASM163280v1. This has the RefSeq accession number of GCF_001632805.1 in the NCBI database.

We will use the esearch tool from E-Direct, and pipe the results into the elink and efetch tools.

In [132]:
!esearch -db assembly -query GCF_001632805.1 | elink -target nucleotide -name assembly_nuccore_refseq | efetch -format fasta_cds_na > data/reference/GCF_001632805_1_cds_from_genomic_unformatted.fa
!head data/reference/GCF_001632805_1_cds_from_genomic_unformatted.fa

>lcl|NZ_CP007220.1_cds_WP_030097698.1_1 [gene=dnaA] [locus_tag=BB28_RS00005] [db_xref=GeneID:31677580] [protein=chromosomal replication initiator protein DnaA] [protein_id=WP_030097698.1] [location=47..1522] [gbkey=CDS]
TTGACTGATGAACTGAATTCCCAGTTCACGGCGGTATGGAATACCGTCGTCGCAGAGCTCAATGGTGACG
ACAACCAATATTTGTCGAGTTTTCCGCCACTCACCCCTCAGCAGCGCGCCTGGCTCACATTGGTCAAGCC
ACTCACCATGGCCGAAGGTTTCGCACTGCTTTCCGTGCCGTCCAGCTTCGTGCAAAACGAGATCGAACGG
CATCTGCGCGGACCCATCGTGGAGGCCCTTTCACGTCGCCTCGGCGAGAACGTCGAACTGGGTGTGCGCA
TCGCGGCACCGGCTCCTGGCGAAGATTTGGAAGTCGCCGGCCCCACGCCCGAGCTCGAGCTCGACGAGGT
CGACGAGACCACCGAGGCGCTTGCTAGCGCACACGAATCGTGGCCGTCGTACTTCATCAACCGTCCGGGC
GGGGCCGACAAGGCCGAGACTCCTGACACGAGCCTCAATGCGCGCTATACCTTCGAATCTTTCGTCATCG
GCGCATCCAACCGGTTCTCGCACGCAGCCGCCGTCGCAGTTTCCGAAGCTCCGGCCCGCGCGTACAACCC
CTTGTTCATCTGGGGTGAATCCGGGTTGGGTAAGACGCATCTACTGCACGCCGCCGGGAACTACGCCCAG


Salmon also recommends using a 'decoy-aware' transcriptome file. This requires a decoy text file with relevant decoy sequences.

More on this can be found in the salmon documentation, but one way to do this is to add the entire genome to the transcript reference above, and have the decoy text file point to it.

We can use edirect, again, to download the genome for M. Chelonae, and bash to add this to the transcript file.

While we are it, we can first format the transcript reference fasta file to use locus tags to identify each sequence.

In [133]:
#download genome file
!esearch -db assembly -query GCF_001632805.1 | elink -target nucleotide -name assembly_nuccore_refseq | efetch -format fasta > data/reference/GCF_001632805_1_genome.fa
!head data/reference/GCF_001632805_1_genome.fa

#reformatting the fasta file to have its header be solely locus id
!sed "s/.*\[locus_tag=/>/" data/reference/GCF_001632805_1_cds_from_genomic_unformatted.fa | sed "s/\] \[.*\CDS\]//"  > data/reference/GCF_001632805_1_cds_from_genomic.fa

#adding genome to end of cds reference for decoy file
!cat data/reference/GCF_001632805_1_cds_from_genomic.fa <(echo) data/reference/GCF_001632805_1_genome.fa > data/reference/GCF_001632805_1_cds_and_decoy.fa

#creating decoy text file with reference to genome
!echo "NZ_CP007220.1" > data/reference/decoys.txt

>NZ_CP007220.1 Mycobacteroides chelonae CCUG 47445 chromosome, complete genome
GGGGCTACTGTCGACGTGTCCCCTACTGCCAGAGAGATGCTCGCCGTTGACTGATGAACTGAATTCCCAG
TTCACGGCGGTATGGAATACCGTCGTCGCAGAGCTCAATGGTGACGACAACCAATATTTGTCGAGTTTTC
CGCCACTCACCCCTCAGCAGCGCGCCTGGCTCACATTGGTCAAGCCACTCACCATGGCCGAAGGTTTCGC
ACTGCTTTCCGTGCCGTCCAGCTTCGTGCAAAACGAGATCGAACGGCATCTGCGCGGACCCATCGTGGAG
GCCCTTTCACGTCGCCTCGGCGAGAACGTCGAACTGGGTGTGCGCATCGCGGCACCGGCTCCTGGCGAAG
ATTTGGAAGTCGCCGGCCCCACGCCCGAGCTCGAGCTCGACGAGGTCGACGAGACCACCGAGGCGCTTGC
TAGCGCACACGAATCGTGGCCGTCGTACTTCATCAACCGTCCGGGCGGGGCCGACAAGGCCGAGACTCCT
GACACGAGCCTCAATGCGCGCTATACCTTCGAATCTTTCGTCATCGGCGCATCCAACCGGTTCTCGCACG
CAGCCGCCGTCGCAGTTTCCGAAGCTCCGGCCCGCGCGTACAACCCCTTGTTCATCTGGGGTGAATCCGG


### STEP 5: Copy data file for Trimmomatic

One of trimmomatics functions is to trim sequence machine specific adapter sequences. These are usually within the trimmomatic installation directory in a folder called adapters.

Directories of packages within conda installations can be confusing, so in the case of using conda with trimmomatic, it may be easier to simply download or create a file with the relevant adapter sequencecs and store it in an easy to find directory.

In [42]:
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa --output TruSeq3-PE.fa
!head TruSeq3-PE.fa 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    95  100    95    0     0   5027      0 --:--:-- --:--:-- --:--:--  5277
>PrefixPE/1
TACACTCTTTCCCTACACGACGCTCTTCCGATCT
>PrefixPE/2
GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT



### STEP 6: Run Trimmomatic
Trimmomatic will trim off any adapter sequences or low quality sequence it detects in the FASTQ files.

For simplicity sake, it is possible to use our original list of run IDs to run trimmomatic on all of our fastq files in one line of code.

In [44]:
!cat data/raw_fastq/list_of_accesionIDS.txt | xargs -I {} trimmomatic PE -threads 2 'data/raw_fastq/{}_1.fastq.gz' 'data/raw_fastq/{}_2.fastq.gz' 'data/trimmed/{}_1_trimmed.fastq.gz' 'data/trimmed/{}_2_trimmed.fastq.gz' 'data/trimmed/{}_1_trimmed_unpaired.fastq.gz'  'data/trimmed/{}_2_trimmed_unpaired.fastq.gz' ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

TrimmomaticPE: Started with arguments:
 -threads 2 data/raw_fastq/SRR13349122_1.fastq.gz data/raw_fastq/SRR13349122_2.fastq.gz data/trimmed/SRR13349122_1_trimmed.fastq.gz data/trimmed/SRR13349122_2_trimmed.fastq.gz data/trimmed/SRR13349122_1_trimmed_unpaired.fastq.gz data/trimmed/SRR13349122_2_trimmed_unpaired.fastq.gz ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 10827590 Both Surviving: 10810267 (99.84%) Forward Only Surviving: 17297 (0.16%) Reverse Only Surviving: 0 (0.00%) Dropped: 26 (0.00%)
TrimmomaticPE: Completed successfully
TrimmomaticPE: Started with arguments:
 -threads 2 data/raw_fastq/SRR13349123_1.fastq.gz data/raw_fastq/SRR13349123_2.fastq.gz data/trimmed/SRR13349123_1_tri

### STEP 7: Run FastQC
FastQC is an invaluable tool that allows you to evaluate whether there are problems with a set of reads. For example, it will provide a report of whether there is any bias in the sequence composition of the reads.

If you notice the results of the trimming, you may have noted the sequences in the reverse reads were few, and largely unpaired. This may be an artifact from how the original sequencing process. This is okay, we can proceed from here simply using the forward reads.

In [46]:
!cat data/raw_fastq/list_of_accesionIDS.txt | xargs -I {} fastqc "data/trimmed/{}_1_trimmed.fastq.gz" -o data/fastqc/


Started analysis of SRR13349122_1_trimmed.fastq.gz
Approx 5% complete for SRR13349122_1_trimmed.fastq.gz
Approx 10% complete for SRR13349122_1_trimmed.fastq.gz
Approx 15% complete for SRR13349122_1_trimmed.fastq.gz
Approx 20% complete for SRR13349122_1_trimmed.fastq.gz
Approx 25% complete for SRR13349122_1_trimmed.fastq.gz
Approx 30% complete for SRR13349122_1_trimmed.fastq.gz
Approx 35% complete for SRR13349122_1_trimmed.fastq.gz
Approx 40% complete for SRR13349122_1_trimmed.fastq.gz
Approx 45% complete for SRR13349122_1_trimmed.fastq.gz
Approx 50% complete for SRR13349122_1_trimmed.fastq.gz
Approx 55% complete for SRR13349122_1_trimmed.fastq.gz
Approx 60% complete for SRR13349122_1_trimmed.fastq.gz
Approx 65% complete for SRR13349122_1_trimmed.fastq.gz
Approx 70% complete for SRR13349122_1_trimmed.fastq.gz
Approx 75% complete for SRR13349122_1_trimmed.fastq.gz
Approx 80% complete for SRR13349122_1_trimmed.fastq.gz
Approx 85% complete for SRR13349122_1_trimmed.fastq.gz
Approx 90% comp

### STEP 8: Run MultiQC
MultiQC reads in the FastQQ reports and generate a compiled report for all the analyzed FASTQ files.

In [47]:
!multiqc -f data/fastqc/



  /// ]8;id=257313;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.11

|           multiqc | MultiQC Version v1.12 now available!
|           multiqc | Search path : /home/jupyter/rnaseq-myco-notebook/data/fastqc
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 26/26  rimmed_fastqc.html
|            fastqc | Found 12 reports
|           multiqc | Compressing plot data
|           multiqc | Deleting    : multiqc_report.html   (-f was specified)
|           multiqc | Deleting    : multiqc_data   (-f was specified)
|           multiqc | Report      : multiqc_report.html
|           multiqc | Data        : multiqc_data
|           multiqc | MultiQC complete


### STEP 9: Index the Transcriptome so that Trimmed Reads Can Be Mapped Using Salmon

In [146]:
!salmon index -t data/reference/GCF_001632805_1_cds_and_decoy.fa -p 8 -i data/reference/transcriptome_index --decoys data/reference/decoys.txt -k 31 --keepDuplicates

Version Info: ### PLEASE UPGRADE SALMON ###
### A newer version of salmon with important bug fixes and improvements is available. ####
###
The newest version, available at https://github.com/COMBINE-lab/salmon/releases
contains new features, improvements, and bug fixes; please upgrade at your
earliest convenience.
###
Sign up for the salmon mailing list to hear about new versions, features and updates at:
https://oceangenomics.com/subscribe
###[2022-03-08 08:11:32.615] [jLog] [info] building index
out : data/reference/transcriptome_index
[2022-03-08 08:11:32.616] [puff::index::jointLog] [info] Running fixFasta

[Step 1 of 4] : counting k-mers

[2022-03-08 08:11:32.845] [puff::index::jointLog] [warning] There were 1 transcripts that would need to be removed to avoid duplicates.
[2022-03-08 08:11:32.846] [puff::index::jointLog] [info] Replaced 0 non-ATCG nucleotides
[2022-03-08 08:11:32.846] [puff::index::jointLog] [info] Clipped poly-A tails from 0 transcripts
wrote 4855 cleaned referen

### STEP 10: Run Salmon to Map Reads to Transcripts and Quantify Expression Levels
Salmon aligns the trimmed reads to the reference transcriptome and generates the read counts per transcript. In this analysis, each gene has a single transcript.

In [147]:
!cat data/raw_fastq/list_of_accesionIDS.txt | xargs -I {} salmon quant -i /home/jupyter/rnaseq-myco-notebook/data/reference/transcriptome_index -l SR -r "data/trimmed/{}_1_trimmed.fastq.gz" -p 8 --validateMappings -o "data/quants/{}_quant"


Version Info: ### PLEASE UPGRADE SALMON ###
### A newer version of salmon with important bug fixes and improvements is available. ####
###
The newest version, available at https://github.com/COMBINE-lab/salmon/releases
contains new features, improvements, and bug fixes; please upgrade at your
earliest convenience.
###
Sign up for the salmon mailing list to hear about new versions, features and updates at:
https://oceangenomics.com/subscribe
###### salmon (selective-alignment-based) v1.5.2
### [ program ] => salmon 
### [ command ] => quant 
### [ index ] => { /home/jupyter/rnaseq-myco-notebook/data/reference/transcriptome_index }
### [ libType ] => { SR }
### [ unmatedReads ] => { data/trimmed/SRR13349122_1_trimmed.fastq.gz }
### [ threads ] => { 8 }
### [ validateMappings ] => { }
### [ output ] => { data/quants/SRR13349122_quant }
Logs will be written to data/quants/SRR13349122_quant/logs
[2022-03-08 08:12:13.496] [jointLog] [info] setting maxHashResizeThreads to 8
[2022-03-08 08:12:

### STEP 11: Report the top 10 most highly expressed genes in the samples

Top 10 most highly expressed genes in each wild-type sample.


In [148]:
!head data/quants/SRR13349122_quant/quant.sf -n 1
!sort -nrk 4,4 data/quants/SRR13349122_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349123_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349124_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349125_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349126_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349127_quant/quant.sf | head -10


Name	Length	EffectiveLength	TPM	NumReads
BB28_RS02220	204	9.377	44475.788787	895.000
BB28_RS20695	231	14.032	36430.207561	1097.000
BB28_RS18745	300	51.326	19546.898386	2153.000
BB28_RS18945	222	12.150	18753.815737	489.000
BB28_RS24750	102	3.543	18017.902665	137.000
BB28_RS09440	300	51.326	15261.651736	1681.000
BB28_RS11370	195	8.348	11220.105496	201.000
BB28_RS14885	195	8.348	11164.284076	200.000
BB28_RS06980	225	12.734	11051.586735	302.000
BB28_RS10735	267	27.299	10531.682101	617.000
sort: write failed: 'standard output': Broken pipe
sort: write error
BB28_RS02220	204	9.377	41409.156034	983.000
BB28_RS20695	231	14.032	34710.457043	1233.000
BB28_RS18745	300	51.326	18978.816872	2466.000
BB28_RS18945	222	12.150	17913.259686	551.000
BB28_RS24750	102	3.543	16388.652308	147.000
BB28_RS09440	300	51.326	13406.771692	1742.000
BB28_RS06980	225	12.734	11695.028413	377.000
BB28_RS14885	195	8.348	11309.437931	239.000
BB28_RS11370	195	8.348	10694.280219	226.000
BB28_RS19155	282	36.744	10589.100692	

Top 10 most highly expressed genes in the double lysogen samples.


In [149]:
!head data/quants/SRR13349122_quant/quant.sf -n 1
!sort -nrk 4,4 data/quants/SRR13349128_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349129_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349130_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349131_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349132_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349133_quant/quant.sf | head -10


Name	Length	EffectiveLength	TPM	NumReads
BB28_RS20695	231	14.032	39694.828923	951.000
BB28_RS02220	204	9.377	30542.732075	489.000
BB28_RS14885	195	8.348	20978.308650	299.000
BB28_RS06975	216	11.099	20475.000251	388.000
BB28_RS21780	213	10.625	17915.968133	325.000
BB28_RS18945	222	12.150	17256.877319	358.000
BB28_RS18745	300	51.326	16922.844977	1483.000
BB28_RS24750	102	3.543	14712.012171	89.000
BB28_RS20665	1293	1043.000	13482.592112	24010.000
BB28_RS13585	237	15.531	11539.702056	306.000
sort: write failed: 'standard output': Broken pipe
sort: write error
BB28_RS20695	231	14.032	38438.774751	1090.000
BB28_RS02220	204	9.377	29340.224685	556.000
BB28_RS14885	195	8.348	22821.786194	385.000
BB28_RS06975	216	11.099	19929.175592	447.000
BB28_RS21780	213	10.625	17325.647754	372.000
BB28_RS18745	300	51.326	17122.406639	1776.000
BB28_RS20665	1293	1043.000	13894.590236	29287.000
BB28_RS18945	222	12.150	13846.745895	340.000
BB28_RS17595	207	9.766	13224.514551	261.000
BB28_RS01170	225	12.734	12940

### STEP 12: Report the expression of a putative acyl-ACP desaturase (BB28_RS16545) that was downregulated in the double lysogen relative to wild-type
A acyl-transferase was reported to be downregulated in the double lysogen as shown in the table of the top 20 upregulated and downregulated genes from the paper describing the study.
![RNA-Seq workflow](images/table-cushman.png)

Use `grep` to report the expression in the wild-type sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [150]:
!grep 'BB28_RS16545' data/quants/SRR13349122_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349123_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349124_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349125_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349126_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349127_quant/quant.sf


BB28_RS16545	987	737.000	283.255057	448.000
BB28_RS16545	987	737.000	238.507427	445.000
BB28_RS16545	987	737.000	293.730999	403.000
BB28_RS16545	987	737.000	275.537174	441.000
BB28_RS16545	987	737.000	312.189685	452.000
BB28_RS16545	987	737.000	290.463575	487.000


Use `grep` to report the expression in the double lysogen sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [151]:
!grep 'BB28_RS16545' data/quants/SRR13349128_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349129_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349130_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349131_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349132_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349133_quant/quant.sf

BB28_RS16545	987	737.000	37.350457	47.000
BB28_RS16545	987	737.000	46.327276	69.000
BB28_RS16545	987	737.000	36.076431	37.000
BB28_RS16545	987	737.000	37.533487	45.000
BB28_RS16545	987	737.000	28.901127	43.000
BB28_RS16545	987	737.000	32.380236	57.000


### STEP 12: Combine Genecounts to a Single Genecount File
Commonly, the readcounts for each sample are combined into a single table, where the rows contain the gene ID, and the columns identify the sample.

In [218]:
##first merge salmon files by number of reads.
!salmon quantmerge --column numreads --quants data/quants/*_quant -o data/quants/merged_quants.txt
##optinally we can rename the columns
!sed -i "1s/.*/Name\tSRR13349122\tSRR13349123\tSRR13349124\tSRR13349125\tSRR13349126\tSRR13349127\tSRR13349128\tSRR13349129\tSRR13349130\tSRR13349131\tSRR13349132\tSRR13349133/" data/quants/merged_quants.txt



Version Info: ### PLEASE UPGRADE SALMON ###
### A newer version of salmon with important bug fixes and improvements is available. ####
###
The newest version, available at https://github.com/COMBINE-lab/salmon/releases
contains new features, improvements, and bug fixes; please upgrade at your
earliest convenience.
###
Sign up for the salmon mailing list to hear about new versions, features and updates at:
https://oceangenomics.com/subscribe
###[2022-03-08 09:38:00.298] [mergeLog] [info] samples: [ data/quants/SRR13349122_quant, data/quants/SRR13349123_quant, data/quants/SRR13349124_quant, data/quants/SRR13349125_quant, data/quants/SRR13349126_quant, data/quants/SRR13349127_quant, data/quants/SRR13349128_quant, data/quants/SRR13349129_quant, data/quants/SRR13349130_quant, data/quants/SRR13349131_quant, data/quants/SRR13349132_quant, data/quants/SRR13349133_quant ]
[2022-03-08 09:38:00.298] [mergeLog] [info] sample names : [ SRR13349122_quant, SRR13349123_quant, SRR13349124_quant, SRR133

### The next tutorial will focus on the analysis of the gene counts

Now that you have read counts per gene, the next tutorial will show you the following workflow that is used to generate the list of differentially expressed genes.

![RNA-Seq workflow](images/count-workflow.png)

If launching from a python framework on Google cloud, the R tutorial will need to use the IRkernel to function, as well as a separate installation of deseq2.

In [ ]:
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c r/label/archive r
!R -e "install.packages('IRkernel',repos = 'http://cran.us.r-project.org')"
!R -e "IRkernel::installspec()"
conda install -c bioconda bioconductor-deseq2

In the following R tutorial we will be merging our gene count file with an feature table reference file.

This can be downloaded by searching the NCBI refseq database, and locating the ftp url:

In [224]:
!curl https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/001/632/805/GCF_001632805.1_ASM163280v1/GCF_001632805.1_ASM163280v1_feature_table.txt.gz --output data/reference/GCF_001632805_1_feature_table.txt.gz 
!gzip -d data/reference/GCF_001632805_1_feature_table.txt.gz

gzip: GCF_001632805_1_feature_table.txt.gz: No such file or directory
